# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
d_group = data.groupby(['CustomerID', 'ProductName'])
t_quantity = pd.DataFrame(d_group.agg({'Quantity':'sum'}))
t_quantity.reset_index()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
pt = pd.pivot_table(data, index = 'ProductName', values = 'Quantity', columns = 'CustomerID', fill_value = 0)
pt

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
ms = squareform(pdist(pt.T, 'euclidean'))
ms

array([[  0.        ,  10.72380529,   9.64365076, ..., 197.94948851,
        218.8241303 , 203.79892051],
       [ 10.72380529,   0.        ,  10.67707825, ..., 197.34994299,
        218.62525014, 203.95342606],
       [  9.64365076,  10.67707825,   0.        , ..., 197.32460566,
        218.02981447, 203.80628057],
       ...,
       [197.94948851, 197.34994299, 197.32460566, ...,   0.        ,
        266.9269563 , 266.9269563 ],
       [218.8241303 , 218.62525014, 218.02981447, ..., 266.9269563 ,
          0.        , 269.25824036],
       [203.79892051, 203.95342606, 203.80628057, ..., 266.9269563 ,
        269.25824036,   0.        ]])

In [17]:
df = pd.DataFrame((1 / (1 + ms)),index = pt.columns, columns = pt.columns)
df

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [20]:
lista_1 = df[200].sort_values(ascending = False)[1:6]
lista_1

CustomerID
1920    0.094414
1072    0.092610
3253    0.090094
3317    0.090094
3909    0.089695
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [21]:
c_equal = t_quantity.loc[lista_1.index]
c_equal

Quantity
CustomerID ProductName                                
1072       Appetizer - Mini Egg Roll, Shrimp         1
           Appetizer - Sausage Rolls                 1
           Beans - Kidney, Canned                    1
           Beef - Montreal Smoked Brisket            1
           Beef - Striploin Aa                       1
...                                                ...
3909       Wine - Charddonnay Errazuriz              2
           Wine - Fume Blanc Fetzer                  1
           Wine - Magnotta, Merlot Sr Vqa            1
           Wine - Toasted Head                       1
           Wine - Valpolicella Masi                  1

[261 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [22]:
r = c_equal.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
r

,Quantity
ProductName,
Soup - Campbells Bean Medley,3
Lamb - Ground,3
Pomello,3
Bay Leaf,3
Pork - Kidney,3
...,...
Kiwi,1
Knife Plastic - White,1
"Lamb - Pieces, Diced",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [25]:
r_2 = r.merge(pt[200], on = 'ProductName')
recommendation = r_2.loc[r_2[200] == 0.0]
recommendation

,Quantity,200
ProductName,,
Soup - Campbells Bean Medley,3,0
Pomello,3,0
Bay Leaf,3,0
Pork - Kidney,3,0
Muffin - Carrot Individual Wrap,3,0
...,...,...
Kellogs Special K Cereal,1,0
Knife Plastic - White,1,0
"Lamb - Pieces, Diced",1,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [ ]:
s = {}
clientes = list(data['CustomerID'].unique())
for x in clientes:
    lista_1 = df[x].sort_values(ascending = False)[1:6]
    c_equal = t_quantity.loc[lista_1.index]
    r = c_equal.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
    r_2 = r.merge(pt[x], on='ProductName')
    recommendation = r_2.loc[r_2[x] == 0.0]
    s[x] = recommendation.head().index

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [31]:
pd.DataFrame(s).T.head()

,0,1,2,3,4
61288,"Mushrooms - Black, Dried","Wine - Magnotta, Merlot Sr Vqa",Chicken - Soup Base,Wine - Chardonnay South,Milk - 1%
77352,Guinea Fowl,Grenadine,Ecolab - Mikroklene 4/4 L,"Oranges - Navel, 72","Coconut - Shredded, Sweet"
40094,"Water - Mineral, Natural",Pasta - Orecchiette,"Oregano - Dry, Rubbed",Quiche Assorted,Beef - Inside Round
23548,Wanton Wrap,Banana Turning,Towels - Paper / Kraft,Bread - Raisin Walnut Oval,Lettuce - Treviso
78981,Lettuce - Frisee,Longos - Chicken Wings,Pop Shoppe Cream Soda,Sprouts - Alfalfa,Beef - Inside Round


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [32]:
nuevo = squareform(pdist(pt.T, 'correlation'))
df_2 = pd.DataFrame(1 / (1 + nuevo))
df_2.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.493805,0.529875,0.515035,0.500570,0.504367,0.487132,0.492406,0.494838,0.500815,...,0.491005,0.486159,0.498339,0.480313,0.497157,0.480750,0.492063,0.485173,0.482914,0.496579
1,0.493805,1.000000,0.500545,0.496849,0.472695,0.483532,0.504988,0.481550,0.500545,0.506509,...,0.508587,0.496849,0.517060,0.506602,0.497813,0.487096,0.528445,0.499846,0.483420,0.484852
2,0.529875,0.500545,1.000000,0.498278,0.517080,0.493067,0.494101,0.481681,0.493067,0.489652,...,0.511597,0.493648,0.496434,0.502956,0.513787,0.526176,0.513045,0.506386,0.510648,0.494672
3,0.515035,0.496849,0.498278,1.000000,0.494727,0.489104,0.494727,0.510742,0.484642,0.490190,...,0.498600,0.489643,0.501690,0.498569,0.495876,0.497664,0.478039,0.507100,0.507023,0.499809
4,0.500570,0.472695,0.517080,0.494727,1.000000,0.494101,0.503354,0.510747,0.489748,0.504478,...,0.485869,0.513118,0.492877,0.499410,0.500310,0.502343,0.482847,0.497874,0.510144,0.522382


In [33]:
nuevo = squareform(pdist(pt.T, 'cityblock'))
df_2 = pd.DataFrame(1 / (1 + nuevo))
df_2.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.008621,0.010638,0.010101,0.009091,0.009615,0.008621,0.009434,0.009259,0.009615,...,0.000565,0.000635,0.000608,0.000712,0.000590,0.000615,0.000581,0.000615,0.000507,0.000582
1,0.008621,1.000000,0.008696,0.008621,0.007519,0.008130,0.008547,0.008264,0.008696,0.009009,...,0.000564,0.000633,0.000608,0.000713,0.000587,0.000612,0.000583,0.000614,0.000505,0.000577
2,0.010638,0.008696,1.000000,0.009259,0.009524,0.009009,0.008696,0.008850,0.009009,0.009009,...,0.000567,0.000635,0.000607,0.000716,0.000592,0.000621,0.000583,0.000618,0.000510,0.000581
3,0.010101,0.008621,0.009259,1.000000,0.008772,0.008929,0.008772,0.010000,0.008772,0.009091,...,0.000565,0.000635,0.000608,0.000715,0.000590,0.000617,0.000578,0.000618,0.000509,0.000582
4,0.009091,0.007519,0.009524,0.008772,1.000000,0.008696,0.008696,0.009524,0.008547,0.009174,...,0.000562,0.000637,0.000605,0.000713,0.000589,0.000616,0.000577,0.000615,0.000509,0.000583


In [35]:
nuevo = squareform(pdist(pt.T, 'cosine'))
df_2 = pd.DataFrame(1 / (1 + nuevo))
df_2.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.532692,0.565443,0.550466,0.538239,0.540192,0.524859,0.527028,0.530712,0.536037,...,0.529019,0.521742,0.534813,0.513650,0.534245,0.516959,0.529475,0.521380,0.523291,0.533971
1,0.532692,1.000000,0.540444,0.536424,0.514712,0.523506,0.546896,0.520122,0.540444,0.545661,...,0.550794,0.536424,0.557488,0.543589,0.539102,0.527408,0.569724,0.540096,0.528386,0.526533
2,0.565443,0.540444,1.000000,0.534784,0.555621,0.529915,0.532824,0.517251,0.529915,0.525866,...,0.550500,0.530173,0.533903,0.537132,0.551752,0.563065,0.551325,0.543480,0.551922,0.533084
3,0.550466,0.536424,0.534784,1.000000,0.533110,0.525642,0.533110,0.545887,0.521187,0.526088,...,0.537266,0.525862,0.538805,0.532472,0.533641,0.534488,0.516147,0.543866,0.547973,0.537860
4,0.538239,0.514712,0.555621,0.533110,1.000000,0.532824,0.544000,0.547992,0.528487,0.542463,...,0.526924,0.551365,0.532274,0.535338,0.540327,0.541355,0.523256,0.536914,0.553549,0.562502


In [36]:
nuevo = squareform(pdist(pt.T, 'jaccard'))
df_2 = pd.DataFrame(1 / (1 + nuevo))
df_2.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.516807,0.535000,0.526570,0.519824,0.520930,0.512712,0.513889,0.515837,0.518692,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,0.516807,1.000000,0.521008,0.518828,0.507463,0.512000,0.524590,0.510204,0.521008,0.523810,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,0.535000,0.521008,1.000000,0.518018,0.529412,0.515419,0.516949,0.508772,0.515419,0.513274,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,0.526570,0.518828,0.518018,1.000000,0.517094,0.513158,0.517094,0.524038,0.510823,0.513393,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,0.519824,0.507463,0.529412,0.517094,1.000000,0.516949,0.523013,0.525114,0.514644,0.522124,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
